In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
#importando datasets http://www.respeitoavida.sp.gov.br/relatorios/
df_obitos_raw=pd.read_csv(r'C:\Users\Renato\OneDrive\Projeto_TERA\Predição Acidentes Motocicleta\Datasets\obitos_publico.csv',encoding='unicode_escape',sep=';')
df_nao_fatais_raw = pd.read_csv(r'C:\Users\Renato\OneDrive\Projeto_TERA\Predição Acidentes Motocicleta\Datasets\acidentes_naofatais.csv',encoding='unicode_escape',sep=';')
df_fatais_raw = pd.read_excel(r'C:\Users\Renato\OneDrive\Projeto_TERA\Predição Acidentes Motocicleta\Datasets\acidentes_fatais.xlsx')

C:\Users\Renato\anaconda3\envs\Data_Science\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
#checando duplicatas
df_obitos_raw.duplicated().value_counts() #achado 97 linhas duplicadas (os outros df's não tem duplicatas)
df_obitos_raw = df_obitos_raw.drop_duplicates()#dropando as 97 duplicatas, temos 34657 linhas

In [4]:
# Criando Dataframes para trabalhar
df_nao_fatais = df_nao_fatais_raw.copy()

df_fatais = df_fatais_raw.copy()#NAO TEMOS COMO FILTRAR POR MOTOCILCETA POIS NAO TEM ESSA COLUNA!!!!

df_obitos = df_obitos_raw.copy()#AQUI CONSEGUIMOS FILTRAR POR MOTOCICLETA

In [5]:
#checando sempre a quantidade de dados dos dataframes
print('obitos',df_obitos.shape)
print('fatais',df_fatais.shape)
print('Ñ fatais',df_nao_fatais.shape)

obitos (34657, 34)
fatais (31791, 32)
Ñ fatais (377613, 55)


In [6]:
#Arrumando o nome das colunas
df_obitos = df_obitos_raw.rename(columns={'LAT_(GEO)':'Latitude',
                                          'LONG_(GEO)':'Longitude',
                                          'Tipo de via':'Tipo de Via',
                                         'Id da Delegacia (RDO)':'Id Delegacia', 'Número do Bo (RDO)':'Número BO',
                                          'Ano do BO (RDO)':'Ano BO','Númeral / KM':'Numero/KM'})

df_nao_fatais = df_nao_fatais_raw.rename(columns={'LAT_(GEO)':'Latitude','LONG_(GEO)':'Longitude'})


df_fatais =df_fatais_raw.rename(columns={'Lat (GEO)':'Latitude','Long (GEO)':'Longitude','Numeral / KM':'Numero/KM',
                                         'Iluminação da via (SIOPM)':'Iluminação',
                                        'Superfície da Via (SIOPM)':'Superfície da via', 
                                         'Dia da semana':'Dia da Semana',
                                         'Tipo de via':'Tipo de Via',
                                         'Tipo de pista (SIOPM)':'Tipo de pista',
                                         'Id Delegacia (RDO)':'Id Delegacia', 'Número BO (RDO)':'Número BO', 
                                         'Ano BO (RDO)':'Ano BO',
                                        'Condições Climáticas (SIOPM)':'Condições Climáticas'})

In [7]:
# dropando colunas duplicada
# no df_nao_fatais, colunas 'Condições da pista' e 'Tipo de pista', portando dropamos a coluna 'Condições da pista'
df_nao_fatais.drop(['Condições da pista'],axis=1,inplace=True)

In [8]:
#alterando tipos de variáveis
df_obitos['Data do Acidente'] = pd.to_datetime(df_obitos['Data do Acidente'],errors='coerce')

df_nao_fatais['Data do Acidente'] = pd.to_datetime(df_nao_fatais['Data do Acidente'],format='%Y-%m-%d')
df_nao_fatais['Ano do Acidente'] = pd.to_datetime(df_nao_fatais['Ano do Acidente'],format='%Y')
df_nao_fatais['Dia do Acidente'] = pd.to_datetime(df_nao_fatais['Dia do Acidente'],format='%d')

df_fatais.loc[31790,'Data do Acidente']='2020-02-27'
df_fatais['Data do Acidente'] = pd.to_datetime(df_fatais['Data do Acidente'],format='%Y-%m-%d')
df_fatais.loc[31790, 'Ano do Acidente']='2020'
df_fatais['Ano do Acidente'] = pd.to_datetime(df_fatais['Ano do Acidente'],format='%Y')
df_fatais['Dia do Acidente'] = pd.to_datetime(df_fatais['Dia do Acidente'],format='%d')

In [9]:
df_obitos.columns

Index(['Id Delegacia', 'Número BO', 'Ano BO', 'Data do Óbito',
       'Data do Acidente', 'Tipo de Via', 'Dia do óbito', 'Mês do Óbito',
       'Mês Óbito Descrição', 'Ano do Óbito', 'Ano/Mês do Óbito',
       'Dia da Semana', 'Hora do Acidente', 'Turno', 'Município',
       'Região Administrativa', 'Logradouro', 'Numero/KM', 'Jurisdição',
       'Administração', 'Conservação', 'Tipo do local da ocorrência',
       'Latitude', 'Longitude', 'Tipo do veículo da vítima', 'Tipo de vítima',
       'Local do óbito', 'Tipo de acidente', 'Sub Tipo do Acidente', 'Sexo',
       'Faixa etária', 'Idade da vítima', 'Outro Veículo Envolvido',
       'Tempo entre o Acidente e o Óbito'],
      dtype='object')

In [10]:
df_obitos[['Hora do Acidente','Data do Acidente']]

,Hora do Acidente,Data do Acidente
0,17:30:00,2021-02-28
1,10:55:00,2021-02-28
2,19:50:00,2021-02-27
3,20:18:00,2021-02-28
4,18:50:00,2021-02-28
...,...,...
34749,19:00:00,2015-01-01
34750,01:20:00,2015-01-01
34751,10:48:00,2015-01-01
34752,07:40:00,2015-01-01


In [11]:
#juntando a df_fatais_geral com df_nao_fatais, criando df_acidentes
df_fatais_geral = df_fatais.merge(df_obitos,
                                  on=['Número BO','Data do Acidente','Ano BO','Hora do Acidente','Turno','Município','Região Administrativa','Jurisdição','Administração','Conservação','Outro Veículo Envolvido','Tipo de Via','Logradouro','Dia da Semana','Numero/KM','Id Delegacia','Latitude','Longitude'])

In [12]:
#checando sempre a quantidade de dados dos dataframes
print('obitos',df_obitos.shape)
print('fatais',df_fatais.shape)
print('Ñ fatais',df_nao_fatais.shape)
print('merge',df_fatais_geral.shape)

obitos (34657, 34)
fatais (31791, 32)
Ñ fatais (377613, 54)
merge (33934, 48)


In [13]:
df_fatais_geral.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33934 entries, 0 to 33933
Data columns (total 48 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   ID                                   33934 non-null  int64         
 1   Id Delegacia                         33934 non-null  int64         
 2   Número BO                            33934 non-null  int64         
 3   Ano BO                               33934 non-null  int64         
 4   Data do Acidente                     33934 non-null  datetime64[ns]
 5   Dia do Acidente                      33934 non-null  datetime64[ns]
 6   Mês do Acidente                      33934 non-null  object        
 7   Ano do Acidente                      33934 non-null  datetime64[ns]
 8   Ano/Mês do Acidente                  33934 non-null  float64       
 9   Dia da Semana                        33934 non-null  object        
 10  Hora do Ac

In [14]:
#checando o pq algumas variáveis acima tem valores nulos. 
df_fatais_geral['Outro Veículo Envolvido'].value_counts()
# Temos apenas 22544 valores não nulos de 33934. Podemos considerar essa diferença de nulos como da categoria "Não Há"?? E os "não disponíveis" podem ser considerados "Não Há"?? Se sim fazer um apply lambda pra corrigir isso. CHECAR COM O GRUPO!!!

AUTOMOVEL         7650
NÃO HÁ            4561
NAO DISPONIVEL    3503
CAMINHAO          3450
MOTOCICLETA       1724
ONIBUS            1376
OUTROS             143
BICICLETA           94
PEDESTRE            43
Name: Outro Veículo Envolvido, dtype: int64

In [15]:
df_fatais_geral['Sub Tipo do Acidente'].value_counts()
#aqui quase 50% dos dados são nulos. E temos muitas categorias....CHECAR COM O GRUPO!!!

NAO DISPONIVEL                           3512
FRONTAL                                  2136
CRUZANDO A VIA                           1858
TRASEIRA                                 1382
LATERAL                                  1144
TOMBAMENTO VEÍCULO DE 2 RODAS            1142
POSTE                                     813
TRANSVERSAL                               782
CAPOTAMENTO                               695
ANDANDO NA VIA / ACOSTAMENTO              523
ÁRVORE                                    464
DEFENSA / BARREIRA                        440
MURO/PAREDE/TALUDE                        406
VEÍCULO ESTACIONADO                       399
OUTROS                                    265
CONDUTOR/PASSAGEIROS FORA DO VEÍCULO      218
QUEDA (ÁGUA, BARRANCO, PONTE)             176
NA CALÇADA                                163
TOMBAMENTO VEÍCULO DE MAIS DE 2 RODAS     154
MEIO-FIO/CALÇAMENTO/DRENAGEM              150
ANIMAL                                    109
ENGAVETAMENTO                     

In [16]:
#criando colunas com labels de fatal e não falal antes de juntar
df_fatais_geral['Acidente Fatal']=1
df_fatais_geral['Acidente Não Fatal']=0
df_nao_fatais['Acidente Fatal']=0
df_nao_fatais['Acidente Não Fatal']=1


In [17]:
#juntando a df_fatais_geral com df_nao_fatais criando df_acidentes
df_acidentes = pd.concat([df_nao_fatais,df_fatais_geral])

In [18]:
df_acidentes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411547 entries, 0 to 33933
Data columns (total 80 columns):
 #   Column                                       Non-Null Count   Dtype         
---  ------                                       --------------   -----         
 0   ID                                           411547 non-null  int64         
 1   Data do Acidente                             411547 non-null  datetime64[ns]
 2   Dia do Acidente                              411547 non-null  datetime64[ns]
 3   Mês do Acidente                              411547 non-null  object        
 4   Ano do Acidente                              411547 non-null  datetime64[ns]
 5   Ano/Mês do Acidente                          411547 non-null  float64       
 6   Dia da Semana                                411547 non-null  object        
 7   Hora do Acidente                             411547 non-null  object        
 8   Turno                                        411547 non-null  obj

In [19]:
#checando sempre a quantidade de dados dos dataframes
print('df_obitos',df_obitos.shape)
print('df_fatais',df_fatais.shape)
print('df_nao_fatais',df_nao_fatais.shape)
print('df_fatais_geral',df_fatais_geral.shape)
print('df_acidentes',df_acidentes.shape)

df_obitos (34657, 34)
df_fatais (31791, 32)
df_nao_fatais (377613, 56)
df_fatais_geral (33934, 50)
df_acidentes (411547, 80)


In [20]:
#FILTROS
# Filtrando por Região metropolitana de SP
df_acidentes_sp = df_acidentes.loc[df_acidentes['Região Administrativa']=='Metropolitana de São Paulo']#
# df_nao_fatais_sp_moto = df_nao_fatais_sp_moto.loc[df_nao_fatais_sp_moto['Data do Acidente']>='2019-01-01']
# df_nao_fatais_sp_moto = df_nao_fatais_sp_moto.loc[df_nao_fatais_sp_moto['Data do Acidente']<='2020-12-31']

In [21]:
#filtrando por acidentes que envolvem motocicleta
# criando dataframes preliminares com diferentes filtros, pra depois concatenar
df_acidentes_sp_moto1 = df_acidentes_sp.loc[df_acidentes_sp['Veículos Envolvidos - Motocicleta']==1] #dados que vem do "não fatais" onde temos motos envolvidas
df_acidentes_sp_moto2 = df_acidentes_sp.loc[df_acidentes_sp['Tipo do veículo da vítima']=='MOTOCICLETA'] # dados que vem dos "fatais", que são os óbitos de motocicleta
df_acidentes_sp_moto3 = df_acidentes_sp.loc[df_acidentes_sp['Outro Veículo Envolvido']=='MOTOCICLETA'] # dados que vem dos "fatais" , que podem ter motos como outo veículo envolvido
#concatenando os 3 dataframes
df_acidentes_sp_moto = pd.concat([df_acidentes_sp_moto1,df_acidentes_sp_moto2,df_acidentes_sp_moto3])
#checando duplicatas
df_acidentes_sp_moto.duplicated().value_counts() #169 duplicatas pois tem como motocicletas o 'Tipo do veículo da vítima' e 'Outro Veículo Envolvido', do df_fatais. deletar!
df_acidentes_sp_moto = df_acidentes_sp_moto.drop_duplicates()

In [22]:
df_acidentes_sp_moto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62479 entries, 0 to 33910
Data columns (total 80 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   ID                                           62479 non-null  int64         
 1   Data do Acidente                             62479 non-null  datetime64[ns]
 2   Dia do Acidente                              62479 non-null  datetime64[ns]
 3   Mês do Acidente                              62479 non-null  object        
 4   Ano do Acidente                              62479 non-null  datetime64[ns]
 5   Ano/Mês do Acidente                          62479 non-null  float64       
 6   Dia da Semana                                62479 non-null  object        
 7   Hora do Acidente                             62479 non-null  object        
 8   Turno                                        62479 non-null  object        


In [23]:
#checando sempre a quantidade de dados dos dataframes
print('df_obitos',df_obitos.shape)
print('df_fatais',df_fatais.shape)
print('df_nao_fatais',df_nao_fatais.shape)
print('df_fatais_geral',df_fatais_geral.shape)
print('df_acidentes',df_acidentes.shape)
print('df_acidentes_sp',df_acidentes_sp.shape)
print('df_acidentes_sp_moto',df_acidentes_sp_moto.shape)


df_obitos (34657, 34)
df_fatais (31791, 32)
df_nao_fatais (377613, 56)
df_fatais_geral (33934, 50)
df_acidentes (411547, 80)
df_acidentes_sp (159898, 80)
df_acidentes_sp_moto (62479, 80)


In [24]:
# checando o balanceamento do dataset
positive_proportion = 100*df_acidentes_sp_moto['Acidente Fatal'].value_counts()[1]/len(df_acidentes_sp_moto)
print('Proporção de fatais:', round(positive_proportion,2),'%','Portanto temos', df_acidentes_sp_moto['Acidente Fatal'].value_counts()[1],'acidentes fatais no dataset.')

Proporção de fatais: 6.91 % Portanto temos 4318 acidentes fatais no dataset.


In [25]:
#PRECISAMOS CHECAR COLUNAS ANTES DO REPLACE NAN
df_acidentes_sp_moto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62479 entries, 0 to 33910
Data columns (total 80 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   ID                                           62479 non-null  int64         
 1   Data do Acidente                             62479 non-null  datetime64[ns]
 2   Dia do Acidente                              62479 non-null  datetime64[ns]
 3   Mês do Acidente                              62479 non-null  object        
 4   Ano do Acidente                              62479 non-null  datetime64[ns]
 5   Ano/Mês do Acidente                          62479 non-null  float64       
 6   Dia da Semana                                62479 non-null  object        
 7   Hora do Acidente                             62479 non-null  object        
 8   Turno                                        62479 non-null  object        


In [26]:
df_acidentes_sp_moto = df_acidentes_sp_moto[['Data do Acidente', 'Dia da Semana',
       'Hora do Acidente', 'Turno', 'Município', 'Jurisdição', 'Administração', 'Conservação',
       'Latitude', 'Longitude', 'Condições Climáticas', 'Iluminação',
       'Mão de direção', 'Relevo', 'Superfície da via',
       'Tipo de pavimento', 'Tipo de pista', 'Traçado',
       'Tipo de Via','Acidente Fatal']]

In [27]:
#Definir variáveis ciclicas:
#mes, dia da semana, hora, turno

#recursive feature extraction (escolhe o número de variáveis)

#Checklist features: Total do dataset 62,5K linhas e 4318 fatais.
# 'Data do Acidente' OK
# 'Dia da Semana' OK
# Turno e hora do acidente : 253 ND e todos vem dos fatais...temos que consertar isso. fillna moda!! JÁ FIZ!
#municipio OK
# Jurisdição: 4 categorias, 882 ND, só no fatais temos 184 ND. fillna randomico e proporcional?
# Administração: 5 categorias, 14020 ND, só no fatais tem 142 ND. fillna randomico e proporcional?
# Conservação: 16 categorias, 1403 ND, só no fatais tem 143 ND
# cond climaticas: 7 categorias, 11529 ND, só nos fatais tem 3588. API de confição do tempo pra completar os dados?*******************
# iluminação : 8 Categorias,  12540 ND, só nos fatias tem 3588. O que faremos aqui?? correlacionar com outros endereços e pegar a informação?*************
# 'Mão de direção' (EXCLUSIVO NAO FATAIS): 2 Categ, 9020 ND, porém não temos dados de fatais. correlacionar com outros endereços e pegar a informação?
# 'Obras na pista': 55k ND, melhor dropar
# 'Relevo' (EXCLUSIVO NAO FATAIS): 3 Categorias, 9020 ND, porém não temos dados de fatais. correlacionar com outros endereços e pegar a informação?
# 'Superfície da via': 6 categorias, 12603 ND,  só nos fatais tem 3583. O que faremos aqui??***********
# Tipo de pavimento (EXCLUSIVO NAO FATAIS): 7 categorias, 9020 ND, porém não temos dados de fatais. correlacionar com outros endereços e pegar a informação?
# Tipo de pista: 3 categorias, 11276 ND, só nos fatais tem 3583 ND. O que fazer aqui?***********
# 'Traçado'(EXCLUSIVO NAO FATAIS):7 categorais, 9020 ND porém nao temos em fatais.
# Veículos envolvidos (EXCLUSIVO NAO FATAIS): todos já são dummies, porém são de não fatais. dropar para prever fatal e não fatal.
# Pessoas envoldias (EXCLUSIVO NAO FATAIS): todos já são dummies, porém são de não fatais. dropar para prever fatal e não fatal.
# Tipo de Via: 2 categorias, 778 ND sendo 129 no fatais.
# Tipo de acidente (EXCLUSIVO NAO FATAIS): todos já são dummies, porém são de não fatais. dropar para prever fatal e não fatal.
# 'Tipo do Local do Acidente' E DEMAIS (EXCLUSIVO PARA FATAIS): DROPAR TUDO PQ É SÓ PRA FATAIS.

In [28]:
df_acidentes_sp_moto[['Condições Climáticas','Latitude','Longitude','Hora do Acidente','Data do Acidente']].sample(20)

,Condições Climáticas,Latitude,Longitude,Hora do Acidente,Data do Acidente
274421,BOM,"-23,735128","-46,5605649",09:42,2019-07-26
194194,BOM,"-23,7114359536","-46,548815941",10:39,2019-12-30
87632,NAO DISPONIVEL,"-23,5778","-46,9066",16:23,2020-09-05
30866,NAO DISPONIVEL,"-23,5466577824","-46,2056692457",07:16:00,2015-06-13
203907,BOM,"-23,6990754035","-46,4376554034",13:27,2019-12-11
13162,BOM,"-23,5543697","-46,6249654",18:46,2021-02-01
356303,CHUVA,"-23,5798280839","-46,6627233133",20:08,2019-02-16
266114,BOM,"-23,59718865","-46,5330500877",20:57,2019-08-12
164088,BOM,"-23,5128128186263","-46,6904486534985",11:06,2020-03-05
114608,BOM,"-23,563833529","-46,7458730389999",22:28,2020-07-10


In [29]:
#entendendo melhor os missing values de condição climática
# df_acidentes_sp_moto['Condições Climáticas'].loc[df_acidentes_sp_moto['Condições Climáticas'].isnull()]
#16 antes de 2015
#700 em 2015
#640 em 2016
#715 em 2017
#770 em 2018
#4550 em 2019
#3646 em 2020
#492 em 2021
df_acidentes_sp_moto['Condições Climáticas'].loc[(df_acidentes_sp_moto['Data do Acidente']>='2000-01-01')&(df_acidentes_sp_moto['Data do Acidente']<'2022-01-01')&(df_acidentes_sp_moto['Condições Climáticas'].isnull())&(df_acidentes_sp_moto['Acidente Fatal']==1)]


Series([], Name: Condições Climáticas, dtype: object)

In [30]:
df_acidentes_sp_moto.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 62479 entries, 0 to 33910
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Data do Acidente      62479 non-null  datetime64[ns]
 1   Dia da Semana         62479 non-null  object        
 2   Hora do Acidente      62479 non-null  object        
 3   Turno                 62479 non-null  object        
 4   Município             62479 non-null  object        
 5   Jurisdição            62479 non-null  object        
 6   Administração         62479 non-null  object        
 7   Conservação           62479 non-null  object        
 8   Latitude              62479 non-null  object        
 9   Longitude             62479 non-null  object        
 10  Condições Climáticas  62479 non-null  object        
 11  Iluminação            62479 non-null  object        
 12  Mão de direção        58161 non-null  object        
 13  Relevo          

In [31]:
#Substituir os nao disponiveis por nulos
df_acidentes_sp_moto.replace('NAO DISPONIVEL', np.nan,inplace=True)
df_acidentes_sp_moto.replace('NÃO', np.nan,inplace=True)
df_acidentes_sp_moto.replace('NÃO HÁ', np.nan,inplace=True)
df_acidentes_sp_moto.columns



Index(['Data do Acidente', 'Dia da Semana', 'Hora do Acidente', 'Turno',
       'Município', 'Jurisdição', 'Administração', 'Conservação', 'Latitude',
       'Longitude', 'Condições Climáticas', 'Iluminação', 'Mão de direção',
       'Relevo', 'Superfície da via', 'Tipo de pavimento', 'Tipo de pista',
       'Traçado', 'Tipo de Via', 'Acidente Fatal'],
      dtype='object')

In [32]:
df_acidentes_sp_moto.shape

(62479, 20)

In [33]:
df_acidentes_sp_moto['Acidente Fatal'].value_counts()


0    58161
1     4318
Name: Acidente Fatal, dtype: int64

In [34]:
#Dropando tudo em branco
# df_acidentes_sp_moto.dropna(inplace=True)

#Dropando quando não temos LAT, LON
df_acidentes_sp_moto = df_acidentes_sp_moto.loc[(df_acidentes_sp_moto['Latitude'].notnull())&(df_acidentes_sp_moto['Longitude'].notnull())]




In [35]:
df_acidentes_sp_moto['Hora do Acidente']

0           02:29
4           13:47
8           21:40
21          15:14
22          15:18
           ...   
32976    11:51:00
33557    14:53:00
33850    14:30:00
33867    15:20:00
33910    22:50:00
Name: Hora do Acidente, Length: 58740, dtype: object

In [36]:
df_acidentes_sp_moto.shape

(58740, 20)

In [37]:
# export
# df_acidentes_sp_moto.to_csv('df_acidentes_sp_moto.csv')

In [38]:
df_acidentes_sp_moto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58740 entries, 0 to 33910
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Data do Acidente      58740 non-null  datetime64[ns]
 1   Dia da Semana         58740 non-null  object        
 2   Hora do Acidente      58573 non-null  object        
 3   Turno                 58573 non-null  object        
 4   Município             58740 non-null  object        
 5   Jurisdição            58648 non-null  object        
 6   Administração         58198 non-null  object        
 7   Conservação           58197 non-null  object        
 8   Latitude              58740 non-null  object        
 9   Longitude             58740 non-null  object        
 10  Condições Climáticas  48219 non-null  object        
 11  Iluminação            47238 non-null  object        
 12  Mão de direção        46452 non-null  object        
 13  Relevo          